In [ ]:
import os
import pathlib
from dotenv import load_dotenv
load_dotenv()

# Install dependencies
LOCAL_DATABRICKS_NOTEBOOK_PATH = os.getenv('LOCAL_DATABRICKS_NOTEBOOK_PATH')
if LOCAL_DATABRICKS_NOTEBOOK_PATH and pathlib.Path(LOCAL_DATABRICKS_NOTEBOOK_PATH).exists():
    print(f"Installing databricks-notebook from {LOCAL_DATABRICKS_NOTEBOOK_PATH}")
    %pip install --editable "{LOCAL_DATABRICKS_NOTEBOOK_PATH}"
else:
    print("Installing databricks-notebook from git")
    %pip install git+https://github.com/datafold/databricks-notebook.git

# Restart to make dependencies available
# %restart_python on databricks notebook
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


In [ ]:
org_token = "zda4wct*ZBF3ybt3vfz" # do not change
identity = None

# We collect basic identity information to help track and resolve any issues
# with SQL translation and provide you with the best experience. This data is
# used internally by Datafold only and helps us:
# - Diagnose translation errors specific to your workspace configuration
# - Improve translation quality based on real usage patterns
# - Provide better support when you need assistance
#
# If you prefer not to share certain information, you can comment out specific
# fields below or remove this entire code block. The tool will still work, but
# we may have limited ability to help troubleshoot issues.

def get_context_info():
    context = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return {
        'workspace_id': context.workspaceId().get(),
        'workspace_url': context.browserHostName().get(),
        'cluster_id': context.clusterId().get(),
        'notebook_path': context.notebookPath().get(),
        'user': context.userName().get()
    }

identity = get_context_info()

In [ ]:
from databricks_notebook import translate_queries_and_render_results

queries_to_translate = [
"""
-- Monthly Sales Analysis Report
-- Analyzes sales trends with category rankings

drop table if exists DF_RESULT_QUERY_1_SQL_2234AC5;
create table DF_RESULT_QUERY_1_SQL_2234AC5 AS (
    WITH monthly_sales_2234AC5 AS (
        SELECT
            PRODUCT_ID,
            PRODUCT_NAME,
            CATEGORY,
            DATE_TRUNC('MONTH', SALE_DATE) AS SALE_MONTH,
            SUM(QUANTITY) AS TOTAL_QUANTITY,
            SUM(T_REVENUE) AS T_REVENUE,
            COUNT(DISTINCT CUSTOMER_ID) AS UNIQUE_CUSTOMERS
        FROM DMA_TRANSLATIONS.DATAFOLD_TMP.SALES_TRANSACTIONS_2234AC5
        GROUP BY
            PRODUCT_ID,
            PRODUCT_NAME,
            CATEGORY,
            DATE_TRUNC('MONTH', SALE_DATE)
    )
    SELECT
        PRODUCT_ID,
        PRODUCT_NAME,
        CATEGORY,
        SALE_MONTH,
        TOTAL_QUANTITY,
        T_REVENUE,
        UNIQUE_CUSTOMERS,
        -- Rank products by t_revenue within each category and month
        RANK() OVER (
            PARTITION BY CATEGORY, SALE_MONTH
            ORDER BY T_REVENUE DESC
        ) AS CATEGORY_RANK
    FROM monthly_sales_2234AC5
    QUALIFY CATEGORY_RANK <= 3
    ORDER BY
        SALE_MONTH DESC,
        CATEGORY,
        T_REVENUE DESC
);
"""
]

translate_queries_and_render_results(queries_to_translate, org_token, identity)